## Урок 5. Part-of-Speech разметка, NER, извлечение отношений


### Тема “POS-tagger и NER”

### Задание 1. Написать теггер на данных с русским языком
проверить UnigramTagger, BigramTagger, TrigramTagger и их комбмнации
написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов
сравнить все реализованные методы сделать выводы
 


In [1]:
!pip install pyconll

In [2]:
!pip install ipymarkup

In [3]:
!pip install corus

In [4]:
!pip install sklearn_crfsuite

In [5]:
#nltk.download() 
import nltk
from nltk.tokenize import word_tokenize
import matplotlib
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

import pyconll
from nltk.tag import DefaultTagger, UnigramTagger, BigramTagger, TrigramTagger
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import classification_report

import os
import corus
!pip install -U spacy
!python -m spacy info
!python -m spacy download ru_core_news_sm
import spacy
from spacy import displacy
import ru_core_news_sm
!pip install slovnet
from navec import Navec
from slovnet import NER
from ipymarkup import show_span_ascii_markup as show_markup
from razdel import tokenize
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from sklearn.metrics import f1_score


============================== Info about spaCy ==============================

spaCy version    3.4.0                         
Location         /Users/Olga/opt/anaconda3/lib/python3.9/site-packages/spacy
Platform         macOS-10.13.6-x86_64-i386-64bit
Python version   3.9.7                         
Pipelines        ru_core_news_sm (3.4.0)       

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 5.9 MB/s eta 0:00:0000:0100:01m
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


In [6]:
full_train = pyconll.load_from_file('ru_syntagrus-ud-train-a.conllu')
full_test = pyconll.load_from_file('ru_syntagrus-ud-dev.conllu')

In [7]:
for sent in full_train[:2]:
    for token in sent:
        print(token.form, token.upos)
    print()

Анкета NOUN
. PUNCT

Начальник NOUN
областного ADJ
управления NOUN
связи NOUN
Семен PROPN
Еремеевич PROPN
был AUX
человек NOUN
простой ADJ
, PUNCT
приходил VERB
на ADP
работу NOUN
всегда ADV
вовремя ADV
, PUNCT
здоровался VERB
с ADP
секретаршей NOUN
за ADP
руку NOUN
и CCONJ
иногда ADV
даже PART
писал VERB
в ADP
стенгазету NOUN
заметки NOUN
под ADP
псевдонимом NOUN
" PUNCT
Муха NOUN
" PUNCT
. PUNCT



In [8]:
fdata_train = []
for sent in full_train[:]:
    fdata_train.append([(token.form, token.upos) for token in sent])
    
fdata_test = []
for sent in full_test[:]:
    fdata_test.append([(token.form, token.upos) for token in sent])
    
fdata_sent_test = []
for sent in full_test[:]:
    fdata_sent_test.append([token.form for token in sent])

In [9]:
MAX_SENT_LEN = max(len(sent) for sent in full_train)
MAX_ORIG_TOKEN_LEN = max(len(token.form) for sent in full_train for token in sent)
print('Наибольшая длина предложения', MAX_SENT_LEN)
print('Наибольшая длина токена', MAX_ORIG_TOKEN_LEN)

Наибольшая длина предложения 194
Наибольшая длина токена 31


In [10]:
len(fdata_train), len(fdata_test), len(fdata_sent_test)

(24516, 8906, 8906)

In [11]:
fdata_train[:5]

[[('Анкета', 'NOUN'), ('.', 'PUNCT')],
 [('Начальник', 'NOUN'),
  ('областного', 'ADJ'),
  ('управления', 'NOUN'),
  ('связи', 'NOUN'),
  ('Семен', 'PROPN'),
  ('Еремеевич', 'PROPN'),
  ('был', 'AUX'),
  ('человек', 'NOUN'),
  ('простой', 'ADJ'),
  (',', 'PUNCT'),
  ('приходил', 'VERB'),
  ('на', 'ADP'),
  ('работу', 'NOUN'),
  ('всегда', 'ADV'),
  ('вовремя', 'ADV'),
  (',', 'PUNCT'),
  ('здоровался', 'VERB'),
  ('с', 'ADP'),
  ('секретаршей', 'NOUN'),
  ('за', 'ADP'),
  ('руку', 'NOUN'),
  ('и', 'CCONJ'),
  ('иногда', 'ADV'),
  ('даже', 'PART'),
  ('писал', 'VERB'),
  ('в', 'ADP'),
  ('стенгазету', 'NOUN'),
  ('заметки', 'NOUN'),
  ('под', 'ADP'),
  ('псевдонимом', 'NOUN'),
  ('"', 'PUNCT'),
  ('Муха', 'NOUN'),
  ('"', 'PUNCT'),
  ('.', 'PUNCT')],
 [('В', 'ADP'),
  ('приемной', 'NOUN'),
  ('его', 'PRON'),
  ('с', 'ADP'),
  ('утра', 'NOUN'),
  ('ожидали', 'VERB'),
  ('посетители', 'NOUN'),
  (',', 'PUNCT'),
  ('-', 'PUNCT'),
  ('кое-кто', 'PRON'),
  ('с', 'ADP'),
  ('важными', 'ADJ'),

In [12]:
unigram_tagger = UnigramTagger(fdata_train)
accuracy_U = unigram_tagger.evaluate(fdata_test)
display(accuracy_U)

0.823732013802982

In [13]:
bigram_tagger = BigramTagger(fdata_train)
accuracy_B = bigram_tagger.evaluate(fdata_test)
display(accuracy_B)

0.6093886320724006

In [14]:
trigram_tagger = TrigramTagger(fdata_train)
accuracy_T = trigram_tagger.evaluate(fdata_test)
display(accuracy_T)

0.1778631421316492

In [15]:
list_1 = [UnigramTagger, BigramTagger]

list_2 = [UnigramTagger, TrigramTagger]

list_3 = [BigramTagger, TrigramTagger]

list_4 = [UnigramTagger, BigramTagger, TrigramTagger]

accuracy_N = []

def backoff_tagger(train_sents, tagger_classes, backoff=None):
    for cls in tagger_classes:
        backoff = cls(train_sents, backoff=backoff)
    return backoff

for list_N in [list_1, list_2, list_3, list_4]:
    backoff = DefaultTagger('NN') 
    tag = backoff_tagger(fdata_train,  
                         list_N,  
                         backoff = backoff) 

    accuracy_N.append(tag.evaluate(fdata_test))

accuracy_N

[0.8275343446838986, 0.8273650628296113, 0.1750309264926102, 0.827905462595221]

In [16]:
import pandas as pd

In [17]:
result = pd.DataFrame({'Tagger': ['Unigram', 'Bigram', 'Trigram', 'Unigram + Bigram', 'UnigramTagger + TrigramTagger', 'BigramTagger + TrigramTagger', 'UnigramTagger+BigramTagger+TrigramTagger'], 'Accuracy' : [accuracy_U, accuracy_B, accuracy_T] + accuracy_N})
result.sort_values('Accuracy', ascending=False)

,Tagger,Accuracy
6,UnigramTagger+BigramTagger+TrigramTagger,0.827905
3,Unigram + Bigram,0.827534
4,UnigramTagger + TrigramTagger,0.827365
0,Unigram,0.823732
1,Bigram,0.609389
2,Trigram,0.177863
5,BigramTagger + TrigramTagger,0.175031


In [18]:
train_tok = [] 
train_label = [] 
train_tok = []
train_label = []
for sent in fdata_train[:]:
    for tok in sent:
        train_tok.append(tok[0])
        train_label.append('NO_TAG' if tok[1] is None else tok[1])
        
test_tok = []
test_label = []
for sent in fdata_test[:]:
    for tok in sent:
        test_tok.append(tok[0])
        test_label.append('NO_TAG' if tok[1] is None else tok[1])

In [19]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(train_label)
test_enc_labels = le.transform(test_label)
le.classes_

array(['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN',
       'NO_TAG', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM',
       'VERB', 'X'], dtype='<U6')

In [20]:
vectorizers = [CountVectorizer(ngram_range=(1, 3), analyzer='char'), 
               TfidfVectorizer(ngram_range=(1, 3), analyzer='char'), 
               HashingVectorizer(ngram_range=(1, 3), analyzer='char', n_features=1000)] 
vectorizers_word = [CountVectorizer(ngram_range=(1, 3), analyzer='word'), 
               TfidfVectorizer(ngram_range=(1, 3), analyzer='word'), 
               HashingVectorizer(ngram_range=(1, 3), analyzer='word', n_features=1000)] 
n_features = [2000, 3000, 5000, 10000]
vectorizers_hash = [HashingVectorizer(ngram_range=(1, 3), analyzer='char', n_features=feat) for feat in n_features]
vectorizers_hash_word = [HashingVectorizer(ngram_range=(1, 3), analyzer='word', n_features=feat) for feat in n_features]
f1_scores = []
accuracy_scores = []

for vectorizer in vectorizers + vectorizers_word + vectorizers_hash + vectorizers_hash_word:
    X_train = vectorizer.fit_transform(train_tok)
    X_test = vectorizer.transform(test_tok[:115000])
    
    lr = LogisticRegression(random_state=0, max_iter=100)
    lr.fit(X_train, train_enc_labels)
    pred = lr.predict(X_test)
    f1 = f1_score(test_enc_labels[:115000], pred, average='weighted')
    f1_scores.append(f1)
    acc = accuracy_score(test_enc_labels[:115000], pred)
    accuracy_scores.append(acc)
    
    print(vectorizer)
    print(classification_report(test_enc_labels[:115000], pred, target_names=le.classes_))

CountVectorizer(analyzer='char', ngram_range=(1, 3))
              precision    recall  f1-score   support

         ADJ       0.92      0.91      0.92     11247
         ADP       0.98      1.00      0.99     10255
         ADV       0.92      0.90      0.91      5986
         AUX       0.81      0.97      0.88      1058
       CCONJ       0.88      0.98      0.93      4276
         DET       0.88      0.75      0.81      2978
        INTJ       0.33      0.36      0.35        11
        NOUN       0.92      0.95      0.94     27241
      NO_TAG       1.00      1.00      1.00       197
         NUM       0.86      0.90      0.88      1436
        PART       0.95      0.78      0.86      3762
        PRON       0.83      0.89      0.86      5346
       PROPN       0.79      0.59      0.67      4315
       PUNCT       1.00      1.00      1.00     21941
       SCONJ       0.81      0.91      0.86      2176
         SYM       1.00      0.68      0.81        53
        VERB       0.94     

In [21]:
result_model = pd.DataFrame({'Vectorizer': vectorizers + vectorizers_word + vectorizers_hash + vectorizers_hash_word,
                            'f1_score': f1_scores})
result_model.sort_values('f1_score', ascending=False)

,Vectorizer,f1_score
0,"CountVectorizer(analyzer='char', ngram_range=(...",0.927836
1,"TfidfVectorizer(analyzer='char', ngram_range=(...",0.921158
9,"HashingVectorizer(analyzer='char', n_features=...",0.903641
8,"HashingVectorizer(analyzer='char', n_features=...",0.901222
7,"HashingVectorizer(analyzer='char', n_features=...",0.897044
6,"HashingVectorizer(analyzer='char', n_features=...",0.895273
2,"HashingVectorizer(analyzer='char', n_features=...",0.882215
4,"TfidfVectorizer(ngram_range=(1, 3))",0.662699
3,"CountVectorizer(ngram_range=(1, 3))",0.656861
13,"HashingVectorizer(n_features=10000, ngram_rang...",0.619406


In [22]:
result_model_acc = pd.DataFrame({'Vectorizer': vectorizers + vectorizers_word + vectorizers_hash + vectorizers_hash_word,
                            'Accuracy': accuracy_scores})
result_model_acc.sort_values('Accuracy', ascending=False)

,Vectorizer,Accuracy
0,"CountVectorizer(analyzer='char', ngram_range=(...",0.929513
1,"TfidfVectorizer(analyzer='char', ngram_range=(...",0.923583
9,"HashingVectorizer(analyzer='char', n_features=...",0.907417
8,"HashingVectorizer(analyzer='char', n_features=...",0.905000
7,"HashingVectorizer(analyzer='char', n_features=...",0.901357
6,"HashingVectorizer(analyzer='char', n_features=...",0.898939
2,"HashingVectorizer(analyzer='char', n_features=...",0.885157
4,"TfidfVectorizer(ngram_range=(1, 3))",0.652983
3,"CountVectorizer(ngram_range=(1, 3))",0.647391
13,"HashingVectorizer(n_features=10000, ngram_rang...",0.628617


In [23]:
result.sort_values('Accuracy', ascending=False).head(1)

,Tagger,Accuracy
6,UnigramTagger+BigramTagger+TrigramTagger,0.827905


In [24]:
result_model.sort_values('f1_score', ascending=False).head(1)

,Vectorizer,f1_score
0,"CountVectorizer(analyzer='char', ngram_range=(...",0.927836


In [25]:
result_model_acc.sort_values('Accuracy', ascending=False).head(1)


,Vectorizer,Accuracy
0,"CountVectorizer(analyzer='char', ngram_range=(...",0.929513


### Задание 2. Проверить насколько хорошо работает NER
данные брать из http://www.labinform.ru/pub/named_entities/
проверить NER из nltk/spacy/deeppavlov
написать свой нер попробовать разные подходы
передаём в сетку токен и его соседей
передаём в сетку только токен
свой вариант
сравнить ваши реализованные подходы на качество (вывести precision/recall/f1_score)


In [26]:
print(os.listdir("collection3"))

['1040.ann', '1047.txt', '289.txt', '504.txt', '517.ann', '271.ann', '262.txt', '276.txt', '20_11_12a.txt', '265.ann', '503.ann', '510.txt', '538.txt', '04_12_12f.ann', '259.ann', '28_11_12g.txt', '2039.ann', '2011.ann', '2002.txt', '2016.txt', '098.ann', '2005.ann', '060.txt', '073.ann', '067.ann', '074.txt', '048.txt', '114.txt', '107.ann', '113.ann', '100.txt', '128.txt', '1134.ann', '1127.txt', '1133.txt', '1120.ann', '1108.ann', '488.ann', '470.txt', 'last_42.txt', 'last_51.ann', '463.ann', 'turkmenija.ann', '305.ann', '316.txt', '302.txt', '311.ann', 'last_45.ann', '477.ann', '464.txt', 'last_56.txt', '339.ann', '458.txt', '459.txt', '338.ann', '310.ann', '303.txt', 'last_57.txt', '465.txt', 'shojgu4.txt', '476.ann', 'last_44.ann', '462.ann', 'last_50.ann', 'last_43.txt', 'shojgu3.ann', '471.txt', '317.txt', '304.ann', '489.ann', '1109.ann', '1121.ann', '1132.txt', '1126.txt', '1135.ann', '129.txt', '101.txt', '112.ann', '106.ann', '115.txt', 'Avtovaz.ann', '049.txt', '075.txt', 

In [27]:
fileDir = r"collection3"
fileExt = r".txt"
documents_txt = [_ for _ in os.listdir(fileDir) if _.endswith(fileExt)]
print(documents_txt)

['1047.txt', '289.txt', '504.txt', '262.txt', '276.txt', '20_11_12a.txt', '510.txt', '538.txt', '28_11_12g.txt', '2002.txt', '2016.txt', '060.txt', '074.txt', '048.txt', '114.txt', '100.txt', '128.txt', '1127.txt', '1133.txt', '470.txt', 'last_42.txt', '316.txt', '302.txt', '464.txt', 'last_56.txt', '458.txt', '459.txt', '303.txt', 'last_57.txt', '465.txt', 'shojgu4.txt', 'last_43.txt', '471.txt', '317.txt', '1132.txt', '1126.txt', '129.txt', '101.txt', '115.txt', '049.txt', '075.txt', '061.txt', '2017.txt', '2003.txt', '28_11_12f.txt', '539.txt', '277.txt', '511.txt', '505.txt', '263.txt', '288.txt', '1046.txt', '1044.txt', '1050.txt', '513.txt', '20_11_12b.txt', '275.txt', '261.txt', '507.txt', '249.txt', '04_12_12b.txt', '2029.txt', '2015.txt', '088.txt', '2001.txt', '077.txt', 'blokhin.txt', '063.txt', '103.txt', '117.txt', '533 (!).txt', '1130.txt', '1124.txt', '498.txt', '1118.txt', 'shojgu6.txt', '467.txt', 'last_55.txt', '301.txt', '315.txt', '473.txt', 'last_41.txt', 'last_69.

In [28]:
text_list = []
for file in documents_txt:
    doc = open('collection3/' + file, encoding='utf-8')
    text = doc.read()
    text_list.append(text)
    
data_text = pd.DataFrame({'text': text_list })
data_text

,text
0,Жириновский предлагает обменять с США Сноудена...
1,Д.Медведев назначил ряд глав региональных МВД\...
2,СМИ: В.Суркову надоело работать в администраци...
3,Д.Медведев освободил от должности еще 10 генер...
4,Д.Медведев уволил командира московского ОМОНа\...
...,...
995,Д.Медведев снял с работы руководителей МВД в 8...
996,Французскую версию Huffington Post возглавила ...
997,Глава швейцарского Центробанка ушел в отставку...
998,"СМИ: Д.Медведев уволил генерала МВД, который п..."


In [29]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /Users/Olga/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/Olga/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [30]:
document = data_text.text[0]

nltk.pos_tag(nltk.word_tokenize(document))


[('Жириновский', 'JJ'),
 ('предлагает', 'NNP'),
 ('обменять', 'NNP'),
 ('с', 'NNP'),
 ('США', 'NNP'),
 ('Сноудена', 'NNP'),
 ('на', 'NNP'),
 ('Бута', 'NNP'),
 ('Лидер', 'NNP'),
 ('ЛДПР', 'NNP'),
 ('Владимир', 'NNP'),
 ('Жириновский', 'NNP'),
 ('предложил', 'NNP'),
 ('обменять', 'NNP'),
 ('бывшего', 'NNP'),
 ('сотрудника', 'NNP'),
 ('ЦРУ', 'NNP'),
 ('США', 'NNP'),
 ('Эдварда', 'NNP'),
 ('Сноудена', 'NNP'),
 (',', ','),
 ('который', 'NNP'),
 ('прибыл', 'NNP'),
 ('в', 'NNP'),
 ('Москву', 'NNP'),
 (',', ','),
 ('на', 'NNP'),
 ('осужденного', 'NNP'),
 ('в', 'NNP'),
 ('Америке', 'NNP'),
 ('бизнесмена', 'NNP'),
 ('Виктора', 'NNP'),
 ('Бута', 'NNP'),
 ('.', '.'),
 ('``', '``'),
 ('Сноудена', 'JJ'),
 ('ни', 'NN'),
 ('в', 'NNP'),
 ('коем', 'NNP'),
 ('случае', 'NNP'),
 ('не', 'NNP'),
 ('высылать', 'NNP'),
 ('в', 'NNP'),
 ('США', 'NNP'),
 (',', ','),
 ('а', 'NNP'),
 ('обменять', 'NNP'),
 ('на', 'NNP'),
 ('Виктора', 'NNP'),
 ('Бута', 'NNP'),
 ('и', 'NNP'),
 ('Константина', 'NNP'),
 ('Ярошенко', 'NN

In [31]:
import nltk
nltk.download('maxent_ne_chunker')
nltk.download('words')


[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /Users/Olga/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /Users/Olga/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [32]:
{(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(document))) if hasattr(chunk, 'label') }

{('Hamilton', 'PERSON'),
 ('Америке', 'PERSON'),
 ('Виктора Бута', 'PERSON'),
 ('Москву', 'PERSON'),
 ('Сноуден', 'PERSON'),
 ('Эдварда Сноудена', 'PERSON'),
 ('Эдварда Сноудена Эдвард Сноуден', 'PERSON')}

In [33]:
pd.read_csv('collection3/001.ann', delimiter='\t')

,T1,LOC 0 6,Россия
0,T2,LOC 50 53,США
1,T3,LOC 57 63,Грузию
2,T4,LOC 87 93,МОСКВА
3,T5,ORG 103 114,РИА Новости
4,T6,LOC 116 122,Россия
5,T7,LOC 141 144,США
6,T8,LOC 161 168,Тбилиси
7,T9,LOC 301 307,России
8,T10,PER 308 324,Григорий Карасин
9,T11,LOC 383 386,США


In [34]:
nlp = ru_core_news_sm.load()
ny_bb = data_text.text[0]
article = nlp(ny_bb)
displacy.render(article, jupyter=True, style='ent')

In [35]:
for token in article:
    print(token.text, token.pos_, token.dep_)

Жириновский PROPN nsubj
предлагает VERB ROOT
обменять VERB xcomp
с ADP case
США PROPN obl
Сноудена PROPN obj
на ADP case
Бута PROPN obl


 SPACE dep
Лидер NOUN nsubj
ЛДПР PROPN nmod
Владимир PROPN appos
Жириновский PROPN flat:name
предложил VERB parataxis
обменять VERB xcomp
бывшего ADJ amod
сотрудника NOUN obj
ЦРУ PROPN nmod
США PROPN nmod
Эдварда PROPN appos
Сноудена PROPN flat:name
, PUNCT punct
который PRON nsubj
прибыл VERB acl:relcl
в ADP case
Москву PROPN obl
, PUNCT punct
на ADP case
осужденного NOUN acl
в ADP case
Америке PROPN obl
бизнесмена NOUN appos
Виктора PROPN appos
Бута PROPN flat:name
. PUNCT punct


 SPACE dep
" PUNCT punct
Сноудена NOUN obj
ни PART advmod
в ADP fixed
коем DET fixed
случае NOUN fixed
не PART advmod
высылать VERB ROOT
в ADP case
США PROPN obl
, PUNCT punct
а CCONJ cc
обменять VERB conj
на ADP case
Виктора PROPN obl
Бута PROPN flat:name
и CCONJ cc
Константина PROPN conj
Ярошенко PROPN flat:name
. PUNCT punct
В ADP case
идеале NOUN ROOT
— PUNCT punct
до

In [36]:
pd.read_csv('collection3/001.ann', delimiter='\t', header=None)

,0,1,2
0,T1,LOC 0 6,Россия
1,T2,LOC 50 53,США
2,T3,LOC 57 63,Грузию
3,T4,LOC 87 93,МОСКВА
4,T5,ORG 103 114,РИА Новости
5,T6,LOC 116 122,Россия
6,T7,LOC 141 144,США
7,T8,LOC 161 168,Тбилиси
8,T9,LOC 301 307,России
9,T10,PER 308 324,Григорий Карасин


In [37]:
!pip install natasha

In [38]:
!pip install navec


In [42]:
text = data_text.text[0]
navec = Navec.load('navec_news_v1_1B_250K_300d_100q.tar')
ner = NER.load('slovnet_ner_news_v1.tar')
ner.navec(navec)

markup = ner(text)
show_markup(markup.text, markup.spans)

Жириновский предлагает обменять с США Сноудена на Бута
                                  LOC PER─────    PER─
Лидер ЛДПР Владимир Жириновский предложил обменять бывшего сотрудника 
      ORG─ PER─────────────────                                       
ЦРУ США Эдварда Сноудена, который прибыл в Москву, на осужденного в 
ORG LOC PER─────────────                   LOC───                   
Америке бизнесмена Виктора Бута.
LOC────            PER───────── 
"Сноудена ни в коем случае не высылать в США, а обменять на Виктора 
 PER─────                                LOC                PER─────
Бута и Константина Ярошенко. В идеале — добавить генерала Олега 
────   PER─────────────────                               PER───
Калугина", — написал он в своем микроблоге в Twitter.
────────                                     ORG──── 
Сноуден, работавший на компанию Booz Allen Hamilton — подрядчика 
PER────                         ORG────────────────              
Центрального разведывательного управ

In [43]:
fileDir = r"collection3"
fileExt = r".ann"
documents_ann = [_ for _ in os.listdir(fileDir) if _.endswith(fileExt)]
print(documents_ann)

['1040.ann', '517.ann', '271.ann', '265.ann', '503.ann', '04_12_12f.ann', '259.ann', '2039.ann', '2011.ann', '098.ann', '2005.ann', '073.ann', '067.ann', '107.ann', '113.ann', '1134.ann', '1120.ann', '1108.ann', '488.ann', 'last_51.ann', '463.ann', 'turkmenija.ann', '305.ann', '311.ann', 'last_45.ann', '477.ann', '339.ann', '338.ann', '310.ann', '476.ann', 'last_44.ann', '462.ann', 'last_50.ann', 'shojgu3.ann', '304.ann', '489.ann', '1109.ann', '1121.ann', '1135.ann', '112.ann', '106.ann', 'Avtovaz.ann', '066.ann', '072.ann', '2004.ann', '099.ann', '2010.ann', '28_11_12a.ann', '2038.ann', '258.ann', '04_12_12g.ann', '264.ann', '502.ann', '516.ann', '270.ann', '1041.ann', '1043.ann', '299.ann', '500.ann', '266.ann', '272.ann', '514.ann', '528.ann', '2006.ann', '2012.ann', '064.ann', '070.ann', '058.ann', '19_11_12h.ann', '110.ann', '104.ann', '138.ann', '1123.ann', '1137.ann', 'last_46.ann', '474.ann', '312.ann', '306.ann', 'shojgu1.ann', 'last_52.ann', '460.ann', '448.ann', '449.ann', 

In [44]:
ann = pd.read_csv('collection3/001.ann', delimiter='\t', header=None)
ann

,0,1,2
0,T1,LOC 0 6,Россия
1,T2,LOC 50 53,США
2,T3,LOC 57 63,Грузию
3,T4,LOC 87 93,МОСКВА
4,T5,ORG 103 114,РИА Новости
5,T6,LOC 116 122,Россия
6,T7,LOC 141 144,США
7,T8,LOC 161 168,Тбилиси
8,T9,LOC 301 307,России
9,T10,PER 308 324,Григорий Карасин


In [58]:
import csv

docs = []
for i in range(len(data_text)):
    words = []
    labels = []
    text = data_text['text'][i]
    
    df = pd.read_csv('collection3/' + documents_ann[i], delimiter='\t', header=None, quoting=csv.QUOTE_NONE, encoding='utf-8')
    df_ann = pd.DataFrame()
    df_ann['Token'] = df.loc[:, 2]
    split_1 = [loc.split() for loc in df.loc[:, 1].values]
    df_ann['Entity'] = [loc[0] for loc in split_1]
       
    dic = {}
    for j in range(len(df)):
        token = df_ann['Token'][j].lower().split()
        entity = df_ann['Entity'][j]
        for tok in token:
            dic[tok] = entity

    for token in tokenize(text):
        if (token.text.lower() in dic.keys()):
            words.append(token.text)
            labels.append(dic[token.text.lower()])
        else:
            words.append(token.text)
            labels.append('OUT')
    
    docs.append([words, labels])

In [59]:
print(docs[0][0]), print(docs[0][1])

['Жириновский', 'предлагает', 'обменять', 'с', 'США', 'Сноудена', 'на', 'Бута', 'Лидер', 'ЛДПР', 'Владимир', 'Жириновский', 'предложил', 'обменять', 'бывшего', 'сотрудника', 'ЦРУ', 'США', 'Эдварда', 'Сноудена', ',', 'который', 'прибыл', 'в', 'Москву', ',', 'на', 'осужденного', 'в', 'Америке', 'бизнесмена', 'Виктора', 'Бута', '.', '"', 'Сноудена', 'ни', 'в', 'коем', 'случае', 'не', 'высылать', 'в', 'США', ',', 'а', 'обменять', 'на', 'Виктора', 'Бута', 'и', 'Константина', 'Ярошенко', '.', 'В', 'идеале', '—', 'добавить', 'генерала', 'Олега', 'Калугина', '"', ',', '—', 'написал', 'он', 'в', 'своем', 'микроблоге', 'в', 'Twitter', '.', 'Сноуден', ',', 'работавший', 'на', 'компанию', 'Booz', 'Allen', 'Hamilton', '—', 'подрядчика', 'Центрального', 'разведывательного', 'управления', 'США', ',', 'в', 'начале', 'июня', 'распространил', 'секретный', 'ордер', 'суда', ',', 'по', 'которому', 'спецслужбы', 'получили', 'доступ', 'ко', 'всем', 'звонкам', 'крупнейшего', 'сотового', 'оператора', 'Verizon'

(None, None)

In [60]:
data, labels = list(zip(*docs))
for w, e in zip(data[0], labels[0]):
    print(f'{w}\t{e}')

Жириновский	OUT
предлагает	OUT
обменять	OUT
с	OUT
США	OUT
Сноудена	OUT
на	OUT
Бута	OUT
Лидер	OUT
ЛДПР	OUT
Владимир	OUT
Жириновский	OUT
предложил	OUT
обменять	OUT
бывшего	OUT
сотрудника	OUT
ЦРУ	OUT
США	OUT
Эдварда	OUT
Сноудена	OUT
,	OUT
который	OUT
прибыл	OUT
в	OUT
Москву	OUT
,	OUT
на	OUT
осужденного	OUT
в	OUT
Америке	OUT
бизнесмена	OUT
Виктора	OUT
Бута	OUT
.	OUT
"	OUT
Сноудена	OUT
ни	OUT
в	OUT
коем	OUT
случае	OUT
не	OUT
высылать	OUT
в	OUT
США	OUT
,	OUT
а	OUT
обменять	OUT
на	OUT
Виктора	OUT
Бута	OUT
и	OUT
Константина	OUT
Ярошенко	OUT
.	OUT
В	OUT
идеале	OUT
—	OUT
добавить	OUT
генерала	OUT
Олега	OUT
Калугина	OUT
"	OUT
,	OUT
—	OUT
написал	OUT
он	OUT
в	OUT
своем	OUT
микроблоге	OUT
в	OUT
Twitter	OUT
.	OUT
Сноуден	OUT
,	OUT
работавший	OUT
на	OUT
компанию	OUT
Booz	OUT
Allen	OUT
Hamilton	OUT
—	OUT
подрядчика	OUT
Центрального	OUT
разведывательного	OUT
управления	OUT
США	OUT
,	OUT
в	OUT
начале	OUT
июня	OUT
распространил	OUT
секретный	OUT
ордер	OUT
суда	OUT
,	OUT
по	OUT
которому	OUT
спецслужбы	OUT

In [61]:
df = pd.DataFrame({'sent_id': [i for j in [[i] * len(s) for i, s in enumerate(data)] for i in j],
                   'data': [i for j in data for i in j],
                   'entities': [i for j in labels for i in j]})
df.head(50)

,sent_id,data,entities
0,0,Жириновский,OUT
1,0,предлагает,OUT
2,0,обменять,OUT
3,0,с,OUT
4,0,США,OUT
5,0,Сноудена,OUT
6,0,на,OUT
7,0,Бута,OUT
8,0,Лидер,OUT
9,0,ЛДПР,OUT


In [62]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s['data'].values.tolist(), 
                                                           s['entities'].values.tolist())]
        self.grouped = self.data.groupby('sent_id').apply(agg_func)
        self.sentences = [s for s in self.grouped]
        
    def get_next(self):
        try: 
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent += 1
            return s 
        except:
            return None

getter = SentenceGetter(df)

sentences = getter.sentences


In [63]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    
    features = {
        'bias': 1.0, 
        'word.lower()': word.lower(), 
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isdigit()': word.isdigit()
    }
    if i > 0:
        word1 = sent[i - 1][0]
        features.update({
            '-1:word.lower()': word1.lower()
        })
    else:
        features['BOS'] = True
    if i < len(sent) - 1:
        word1 = sent[i + 1][0]
        features.update({
            '+1:word.lower()': word1.lower()
        })
    else:
        features['EOS'] = True

    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, label in sent]

def sent2tokens(sent):
    return [token for token, label in sent]


In [64]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]

In [65]:
X[2][7]

{'bias': 1.0,
 'word.lower()': 'в',
 'word[-3:]': 'в',
 'word[-2:]': 'в',
 'word.isdigit()': False,
 '-1:word.lower()': 'работать',
 '+1:word.lower()': 'администрации'}

In [66]:
len(X)

1000

In [67]:
X_train = X[:700]
X_test = X[700:]
y_train = y[:700]
y_test = y[700:]

In [68]:
%%time

crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', #Градиентный спуск с использованием метода L-BFGS
    c1=0.1, #Коэффициент для регуляризации L1
    c2=0.1, #Коэффициент для регуляризации L2
    max_iterations=1000, #Максимальное количество итераций
    all_possible_transitions=True, #Генерация объектов (не встречающихся в обучающих данных)
    verbose=True #Включение режима тренировки
)

try:
    crf.fit(X_train, y_train)
except AttributeError:
    pass

loading training data to CRFsuite: 100%|██████████| 700/700 [00:04<00:00, 162.83it/s]



Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 85907
Seconds required: 0.964

L-BFGS optimization
c1: 0.100000
c2: 0.100000
num_memories: 6
max_iterations: 1000
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=0.22  loss=65101.15 active=85833 feature_norm=1.00
Iter 2   time=0.11  loss=11866.86 active=85408 feature_norm=2.09
Iter 3   time=0.12  loss=11479.92 active=85618 feature_norm=2.15
Iter 4   time=0.11  loss=10762.37 active=41710 feature_norm=2.36
Iter 5   time=0.11  loss=10578.59 active=29199 feature_norm=2.47
Iter 6   time=0.12  loss=10362.50 active=19895 feature_norm=2.60
Iter 7   time=0.11  loss=9836.94  active=14718 feature_norm=2.78
Iter 8   time=0.23  loss=8791.67  active=11988 feature_norm=3.95
Iter 9   time=0.12  loss=8383.66  active=13726 feature_norm=4.18
Iter 10  time

In [69]:
all_entities = sorted(df.entities.unique().tolist())
len(all_entities)

4

In [70]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=all_entities)


0.9840980299899303

In [72]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    
    features = {
        'bias': 1.0, 
        'word.lower()': word.lower(), 
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isdigit()': word.isdigit()
    }
    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, label in sent]

def sent2tokens(sent):
    return [token for token, label in sent]

In [73]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]

In [74]:
X[1][7]

{'bias': 1.0,
 'word.lower()': 'мвд',
 'word[-3:]': 'МВД',
 'word[-2:]': 'ВД',
 'word.isdigit()': False}

In [75]:
len(X)

1000

In [76]:
X_train = X[:700]
X_test = X[700:]
y_train = y[:700]
y_test = y[700:]

In [77]:
%%time

crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=1000,
    all_possible_transitions=True,
    verbose=True
)

try:
    crf.fit(X_train, y_train)
except AttributeError:
    pass

loading training data to CRFsuite: 100%|██████████| 700/700 [00:02<00:00, 334.36it/s]



Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 32191
Seconds required: 0.461

L-BFGS optimization
c1: 0.100000
c2: 0.100000
num_memories: 6
max_iterations: 1000
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=0.15  loss=65495.05 active=32188 feature_norm=1.00
Iter 2   time=0.08  loss=11793.74 active=32135 feature_norm=2.11
Iter 3   time=0.08  loss=11438.25 active=32123 feature_norm=2.17
Iter 4   time=0.08  loss=10757.28 active=16357 feature_norm=2.38
Iter 5   time=0.08  loss=10582.87 active=11707 feature_norm=2.48
Iter 6   time=0.09  loss=10368.85 active=7816  feature_norm=2.61
Iter 7   time=0.09  loss=9832.85  active=6076  feature_norm=2.80
Iter 8   time=0.19  loss=8895.44  active=5101  feature_norm=3.83
Iter 9   time=0.13  loss=8488.73  active=4865  feature_norm=4.12
Iter 10  time

In [78]:
all_entities = sorted(df.entities.unique().tolist())
len(all_entities)

4

In [79]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=all_entities)

0.9843322190084342